<a href="https://colab.research.google.com/github/HaoyueFeng/fictional-system-story/blob/master/Assignment_of_humor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np
import os
import pandas as pd


In [8]:
from google.colab import files 
import io 


Saving train.csv to train (1).csv


In [9]:
uploaded = files.upload()
inputs_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))


uploaded_test = files.upload()
inputs_test = pd.read_csv(io.BytesIO(uploaded_test['test.csv']))

Saving test.csv to test (1).csv


In [10]:
def modifier (inputs):
    text_orig = inputs['original'].str.replace('<', '', regex=False).str.replace('/>', '', regex=False)
    tmp = inputs['original'].str.split('<.*\\>')
    text_modified = tmp.str[0] + inputs['edit'] + tmp.str[1]
    grades = inputs['meanGrade']

    texts = text_modified.tolist()
    humor = grades.tolist()

    index = np.arange(len(texts))
    np.random.shuffle(index)
    texts = np.array(texts)[index]
    humor = np.array(humor)[index]
    
    return texts,humor

In [11]:
MAX_NB_WORDS = 20000
EMBEDDING_DIM=100
hidden_dim=4

In [12]:
texts,humor = modifier(inputs_train)
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [13]:
MAX_SEQUENCE_LENGTH = max(len(seq) for seq in sequences)

word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

x_train = data
y_train = humor

In [14]:
from keras.layers import Embedding, Input, LSTM, Dense
from keras.models import Model

In [15]:
embeddings_index = {}
"""
f = open(os.path.join('glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
"""
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
input_ = Input(shape=(20,))
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False) (input_)
rnn = LSTM(hidden_dim, return_sequences=False, return_state=False)(embedding_layer)
result = Dense(1)(rnn)


model = Model(input_, result)
model.compile(loss='mean_squared_error',
              optimizer='adam')
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 100)           1045600   
_________________________________________________________________
lstm (LSTM)                  (None, 4)                 1680      
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 1,047,285
Trainable params: 1,685
Non-trainable params: 1,045,600
_________________________________________________________________


In [17]:
model.fit(x_train, y_train,
          epochs=2, batch_size=1)

Epoch 1/2
1005/9652 [==>...........................] - ETA: 38s - loss: 0.3997

KeyboardInterrupt: ignored

In [18]:
#Testing

tests,humor = modifier(inputs_test)
test_seqs = tokenizer.texts_to_sequences(tests)
MAX_SEQUENCE_LENGTH = max(len(seq) for seq in test_seqs)
x_test = pad_sequences(test_seqs, maxlen=MAX_SEQUENCE_LENGTH)
prediction = model.predict(x_test)


## **SUB-TASK 2**

In [19]:
uploaded_train = files.upload()
inputs_dual = pd.read_csv(io.BytesIO(uploaded_train['train.csv']))

Saving train.csv to train (2).csv


In [20]:
text_orig_1 = inputs_dual['original1'].str.replace('<', '', regex=False).str.replace('/>', '', regex=False)
text_orig_2 = inputs_dual['original2'].str.replace('<', '', regex=False).str.replace('/>', '', regex=False)
tmp_1 = inputs_dual['original1'].str.split('<.*\\>')
tmp_2 = inputs_dual['original2'].str.split('<.*\\>')
text_modified_1 = tmp_1.str[0] + inputs_dual['edit1'] + tmp_1.str[1]
text_modified_2 = tmp_2.str[0] + inputs_dual['edit2'] + tmp_2.str[1]
grades_1 = inputs_dual['meanGrade1']
grades_2 = inputs_dual['meanGrade2']

texts_dual = text_modified_1.tolist() + text_modified_2.to_list()
humor_dual = grades_1.tolist() + grades_2.tolist()

index_dual = np.arange(len(texts_dual))
np.random.shuffle(index_dual)
texts_dual = np.array(texts_dual)[index_dual]
humor_dual = np.array(humor_dual)[index_dual]



In [21]:
sequences_dual = tokenizer.texts_to_sequences(texts_dual)
data_dual = pad_sequences(sequences_dual, maxlen=MAX_SEQUENCE_LENGTH)

x_train_dual = data_dual
y_train_dual = humor_dual

In [22]:
uploaded_test = files.upload()
inputs_test = pd.read_csv(io.BytesIO(uploaded_test['test.csv']))

Saving test.csv to test (2).csv


In [23]:
text_orig_1 = inputs_dual['original1'].str.replace('<', '', regex=False).str.replace('/>', '', regex=False)
text_orig_2 = inputs_dual['original2'].str.replace('<', '', regex=False).str.replace('/>', '', regex=False)
tmp_1 = inputs_dual['original1'].str.split('<.*\\>')
tmp_2 = inputs_dual['original2'].str.split('<.*\\>')
text_modified_1 = tmp_1.str[0] + inputs_dual['edit1'] + tmp_1.str[1]
text_modified_2 = tmp_2.str[0] + inputs_dual['edit2'] + tmp_2.str[1]
grades_1 = inputs_dual['meanGrade1']
grades_2 = inputs_dual['meanGrade2']


seq_test_left = tokenizer.texts_to_sequences(text_modified_1.tolist())
x_test_left = pad_sequences(seq_test_left, maxlen=MAX_SEQUENCE_LENGTH)
seq_test_right = tokenizer.texts_to_sequences(text_modified_2.tolist())
x_test_right = pad_sequences(seq_test_right, maxlen=MAX_SEQUENCE_LENGTH)
 
y_test_left = grades_1.tolist()
y_test_right = grades_2.tolist()

In [24]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

NameError: ignored

In [25]:
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False) (input_)
rnn = LSTM(hidden_dim, return_sequences=False, return_state=False)(embedding_layer)
# flat = Flatten() (rnn)
result = Dense(1)(rnn)


model = Model(input_, result)
model.compile(loss='mean_squared_error',
              optimizer='adam')
model.summary()


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25, 100)           1045600   
_________________________________________________________________
lstm_1 (LSTM)                (None, 4)                 1680      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 1,047,285
Trainable params: 1,685
Non-trainable params: 1,045,600
_________________________________________________________________


In [26]:
model.fit(x_train_dual, y_train_dual,
          epochs=1, batch_size=100)

188/188 [==============================] - 2s 10ms/step - loss: 0.5774


In [32]:

preds_left = model.predict(x_test_left)
preds_right = model.predict(x_test_right)
dual_pred = (preds_left > preds_right).reshape(-1)
dual_true = (np.array(y_test_left) > np.array(y_test_right)).reshape(-1)


array([ True, False,  True, ..., False, False,  True])